In [1]:
import os
import shutil

In [2]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)


In [3]:
from codes.tests.TestsClass import TrainModel, TestSet

2024-02-22 12:10:44.550812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 12:10:44.550919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 12:10:44.553360: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 12:10:59.706034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
TrainModel.get_res_instance_test(247, actual_test=True, add=False)

/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/logs/runs/SNPS/6/rs12203592/transformer/transformer_V2/Paul/247_baseline_model_focal/


In [ ]:
TestSet.get_analyse_res_test_set(33, actual_test=True)